In [8]:
pip install opencv-python mediapipe msvc-runtime

SyntaxError: invalid syntax (<ipython-input-8-840f382e1e4b>, line 1)

In [9]:
pip install mediapipe

SyntaxError: invalid syntax (<ipython-input-9-e33e4ab0c104>, line 1)

In [10]:
import cv2
import time
import mediapipe as mp

TypeError: required field "type_ignores" missing from Module

In [11]:
# Grabbing the Holistic Model from Mediapipe and
# Initializing the Model
mp_holistic = mp.solutions.holistic
holistic_model = mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
 
# Initializing the drawing utils for drawing the facial landmarks on image
mp_drawing = mp.solutions.drawing_utils

TypeError: required field "type_ignores" missing from Module

In [12]:
# #Gemini Test
# %pip install gemini

In [13]:
# import gemini
# response = gemini.generate_text(prompt="Tell me a joke")
# print(response)

In [14]:
#WORKING for Groq
import cv2
import mediapipe as mp
import re
from groq import Groq
import time
import os

# Initialize MediaPipe components
mp_holistic = mp.solutions.holistic

# Create a holistic model instance
holistic_model = mp_holistic.Holistic()

# Video capture from the default camera
capture = cv2.VideoCapture(0)

# Define frame-skip rate (e.g., process 1 out of every 'n' frames)
frame_skip = 2
frame_count = 0

# Initialize API client
key = ""
client = Groq(api_key=key)

# Variable to store ASL letters
result = ""

while capture.isOpened():
    ret, frame = capture.read()

    if not ret:
        break

    # Increment the frame count
    frame_count += 1

    # Skip frames based on the defined frame_skip rate
    if frame_count % frame_skip != 0:
        continue

    # Converting the frame from BGR to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Making predictions using holistic model
    image.flags.writeable = False
    results = holistic_model.process(image)
    image.flags.writeable = True

    hand_joints = []

    # Extracting right hand landmarks if available
    if results.right_hand_landmarks:
        right_hand_joints = [
            (id, lm.x, lm.y, lm.z)
            for id, lm in enumerate(results.right_hand_landmarks.landmark)
        ]
        hand_joints.append(right_hand_joints)

    # Extracting left hand landmarks if available
    if results.left_hand_landmarks:
        left_hand_joints = [
            (id, lm.x, lm.y, lm.z)
            for id, lm in enumerate(results.left_hand_landmarks.landmark)
        ]
        hand_joints.append(left_hand_joints)

    # Prepare content for the API request
    if hand_joints:
        content = (
            f"{hand_joints} Each of these points is a number followed by a 3D vector "
            f"(number, x, y, z), each of the numbers corresponds to a joint on the hand. "
            f"0. WRIST 1. THUMB_CMC 2. THUMB_MCP 3. THUMB_IP 4. THUMB_TIP "
            f"5. INDEX_FINGER_MCP 6. INDEX_FINGER_PIP 7. INDEX_FINGER_DIP 8. INDEX_FINGER_TIP "
            f"9. MIDDLE_FINGER_MCP 10. MIDDLE_FINGER_PIP 11. MIDDLE_FINGER_DIP 12. MIDDLE_FINGER_TIP "
            f"13. RING_FINGER_MCP 14. RING_FINGER_PIP 15. RING_FINGER_DIP 16. RING_FINGER_TIP "
            f"17. PINKY_MCP 18. PINKY_PIP 19. PINKY_DIP 20. PINKY_TIP. "
            f"What ASL letter does this create? Please return the answer in the format 'ASL Letter: 'X''."
        )

        try:
            completion = client.chat.completions.create(
                model="llama-3.2-90b-vision-preview",
                messages=[{"role": "user", "content": content}],
                temperature=0.5,
                max_tokens=500,
                top_p=1,
                stream=False,
            )

            text = completion.choices[0].message.content
            #print(text)

            # Extract ASL letter from the API response
            # ASL_letter_match = re.search(r'ASL Letter: ["\']([A-Z])["\']', text)
            # if ASL_letter_match:
            #     asl_letter = ASL_letter_match.group(1)
            #     result += asl_letter
            #     print(result)  # Print the updated result

            ASL_letter_match = re.search(r'ASL Letter:\s*["\']?([A-Z])["\']?', text)

            if ASL_letter_match:
                asl_letter = ASL_letter_match.group(1)
                result += asl_letter
                print(result)
            else:
                print("No ASL letter found in the response.")

        except Exception as e:
            print(f"Error with API response: {e}")

    time.sleep(2)

    # Enter key 'q' to break the loop
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

# Release the capture and destroy windows
capture.release()
cv2.destroyAllWindows()

TypeError: required field "type_ignores" missing from Module

In [ ]:
# # (joint, x, y, z) with frame-skip

# import cv2
# import mediapipe as mp
# from groq import Groq
# import re

# # Initialize MediaPipe components
# mp_holistic = mp.solutions.holistic
# mp_drawing = mp.solutions.drawing_utils

# # Create a holistic model instance
# holistic_model = mp_holistic.Holistic()

# # (0) in VideoCapture is used to connect to your computer's default camera
# capture = cv2.VideoCapture(0)

# # Define frame-skip rate (e.g., process 1 out of every 'n' frames)
# frame_skip = 5
# frame_count = 0

# result = ""

# while capture.isOpened():
#     # Capture frame by frame
#     ret, frame = capture.read()

#     if not ret:
#         break

#     # Increment the frame count
#     frame_count += 1

#     # Skip frames based on the defined frame_skip rate
#     if frame_count % frame_skip != 0:
#         continue

#     # Converting the frame from BGR to RGB
#     image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

#     # Making predictions using holistic model
#     image.flags.writeable = False
#     results = holistic_model.process(image)
#     image.flags.writeable = True

#     hand_joints = []

#     # Extracting right hand landmarks if available
#     if results.right_hand_landmarks:
#         right_hand_joints = []
#         for id, lm in enumerate(results.right_hand_landmarks.landmark):
#             right_hand_joints.append((id, lm.x, lm.y, lm.z))  # Store the landmark ID and normalized coordinates
#         hand_joints.append(right_hand_joints)
#         #print("Right Hand Joints:", right_hand_joints)

#     # Extracting left hand landmarks if available
#     if results.left_hand_landmarks:
#         left_hand_joints = []
#         for id, lm in enumerate(results.left_hand_landmarks.landmark):
#             left_hand_joints.append((id, lm.x, lm.y, lm.z))  # Store the landmark ID and normalized coordinates
#         hand_joints.append(left_hand_joints)
#         #print("Left Hand Joints:", left_hand_joints)

#     content = str(hand_joints) + "Each of these points is a number followed by a 3 dimensional vector in space (number, x, y, z), each of the numbers corresponds to a joint on the hand. 0. WRIST 1. THUMB_CMC 2. THUMB_MCP  3. THUMB_IP 4. THUMB_TIP 5. INDEX FINGER_MCP 6. INDEX FINGER PIP 7. INDEX FINGER_DIP 8. INDEX FINGER TIP 9. MIDDLE FINGER_MCP 10. MIDDLE FINGER_PIP 11. MIDDLE FINGER DIP 12. MIDDLE FINGER TIP 13. RING FINGER MCP 14. RING FINGER PIP 15. RING FINGER_DIP 16. RING FINGER TIP 17. PINKY MCP 18. PINKY PIP 19. PINKY DIP 20. PINKY TIP What ASL letter does this create. Please return the answer in the format 'ASL Letter: 'X''."
    
#     if len(hand_joints) > 0: 
#         GROQ_API_KEY = ""
#         client = Groq(api_key=GROQ_API_KEY)
#         completion = client.chat.completions.create(
#             model="llama-3.2-90b-vision-preview",
#             messages=[
#                 {
#                     "role": "user",
#                     "content": content
#                 }
#             ],
#             temperature=1,
#             max_tokens=1024,
#             top_p=1,
#             stream=False,
#             stop=None,
#         )
#         #print(completion.choices[0].message)
        
#         text = completion.choices[0].message.content
#         #print(text + "\n")
        
#         ASL_letter = re.search(r'ASL Letter: ["\']([A-Z])["\']', text)
#         if ASL_letter:
#             asl_letter = ASL_letter.group(1)
#             print(asl_letter)
    
#         # if len(capital_letters) == 1: 
#         #     final_letters += capital_letters
#         # elif len(capital_letters) > 1:
#         #     unique_letters = []
#         #     for letter in capital_letters:
#         #         if letter.isalpha() and letter not in unique_letters:
#         #                 unique_letters.append(letter)
#         #     final_letters += unique_letters

#         # if ASL_letter:
#         #     result = result + ASL_letter
#         #     print(result)
        
#         hand_joints.clear()

#     # Enter key 'q' to break the loop
#     if cv2.waitKey(5) & 0xFF == ord('q'):
#         break

# # When all the process is done
# # Release the capture
# capture.release()
# cv2.destroyAllWindows()

In [ ]:
# # (joint, x, y, z)
# import cv2
# import time
# import mediapipe as mp
# from groq import Groq

# # Initialize MediaPipe components
# mp_holistic = mp.solutions.holistic
# mp_drawing = mp.solutions.drawing_utils

# # Create a holistic model instance
# holistic_model = mp_holistic.Holistic()

# # (0) in VideoCapture is used to connect to your computer's default camera
# capture = cv2.VideoCapture(0)

# while capture.isOpened():
#     # Capture frame by frame
#     ret, frame = capture.read()

#     # Converting the frame from BGR to RGB
#     image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

#     # Making predictions using holistic model
#     image.flags.writeable = False
#     results = holistic_model.process(image)
#     image.flags.writeable = True

#     hand_joints = []

#     # Extracting right hand landmarks if available
#     if results.right_hand_landmarks:
#         right_hand_joints = []
#         for id, lm in enumerate(results.right_hand_landmarks.landmark):
#             right_hand_joints.append((id, lm.x, lm.y, lm.z))  # Store the landmark ID and normalized coordinates
#         hand_joints.append(right_hand_joints)
#         #print("Right Hand Joints:", right_hand_joints)

#     # Extracting left hand landmarks if available
#     if results.left_hand_landmarks:
#         left_hand_joints = []
#         for id, lm in enumerate(results.left_hand_landmarks.landmark):
#             left_hand_joints.append((id, lm.x, lm.y, lm.z))  # Store the landmark ID and normalized coordinates
#         hand_joints.append(left_hand_joints)
#         print("Left Hand Joints:", left_hand_joints)
    
#     # Enter key 'q' to break the loop
#     if cv2.waitKey(5) & 0xFF == ord('q'):
#         break

# # When all the process is done
# # Release the capture
# capture.release()
# cv2.destroyAllWindows()

In [ ]:
# from groq import Groq

#     content = str(hand_joints) + "\n\neach of these points is a number followed by a 3 dimensional vector in space (number, x, y, z), each of the numbers corresponds to a joint on the hand. \n\n0. WRIST 1. THUMB_CMC 2. THUMB_MCP  3. THUMB_IP 4. THUMB_TIP 5. INDEX FINGER_MCP 6. INDEX FINGER PIP 7. INDEX FINGER_DIP 8. INDEX FINGER TIP 9. MIDDLE FINGER_MCP 10. MIDDLE FINGER_PIP 11. MIDDLE FINGER DIP 12. MIDDLE FINGER TIP 13. RING FINGER MCP 14. RING FINGER PIP 15. RING FINGER_DIP 16. RING FINGER TIP 17. PINKY MCP 18. PINKY PIP 19. PINKY DIP 20. PINKY TIP\n\nwhat ASL letter does this create. Please return just the letter."

#     GROQ_API_KEY = ""
#     client = Groq(api_key=GROQ_API_KEY)
#     completion = client.chat.completions.create(
#         model="llama-3.2-90b-vision-preview",
#         messages=[
#             {
#                 "role": "user",
#                 "content": content
#             }
#         ],
#         temperature=1,
#         max_tokens=1024,
#         top_p=1,
#         stream=False,
#         stop=None,
#     )
#     print(completion.choices[0].message)


In [5]:
# # (0) in VideoCapture is used to connect to your computer's default camera
# capture = cv2.VideoCapture(0)

# # Initializing current time and precious time for calculating the FPS
# previousTime = 0
# currentTime = 0

# while capture.isOpened():
# 	# capture frame by frame
# 	ret, frame = capture.read()

# 	# resizing the frame for better view
# 	frame = cv2.resize(frame, (800, 600))

# 	# Converting the from BGR to RGB
# 	image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

# 	# Making predictions using holistic model
# 	# To improve performance, optionally mark the image as not writeable to
# 	# pass by reference.
# 	image.flags.writeable = False
# 	results = holistic_model.process(image)
# 	image.flags.writeable = True

# 	# Converting back the RGB image to BGR
# 	image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

# 	# Drawing the Facial Landmarks
# 	mp_drawing.draw_landmarks(
# 	image,
# 	results.face_landmarks,
# 	mp_holistic.FACEMESH_CONTOURS,
# 	mp_drawing.DrawingSpec(
# 		color=(255,0,255),
# 		thickness=1,
# 		circle_radius=1
# 	),
# 	mp_drawing.DrawingSpec(
# 		color=(0,255,255),
# 		thickness=1,
# 		circle_radius=1
# 	)
# 	)

# 	# Drawing Right hand Land Marks
# 	mp_drawing.draw_landmarks(
# 	image, 
# 	results.right_hand_landmarks, 
# 	mp_holistic.HAND_CONNECTIONS
# 	)

# 	# Drawing Left hand Land Marks
# 	mp_drawing.draw_landmarks(
# 	image, 
# 	results.left_hand_landmarks, 
# 	mp_holistic.HAND_CONNECTIONS
# 	)
	
# 	# Calculating the FPS
# 	currentTime = time.time()
# 	fps = 1 / (currentTime-previousTime)
# 	previousTime = currentTime
	
# 	# Displaying FPS on the image
# 	cv2.putText(image, str(int(fps))+" FPS", (10, 70), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)

# 	# Display the resulting image
# 	cv2.imshow("Facial and Hand Landmarks", image)

# 	# Enter key 'q' to break the loop
# 	if cv2.waitKey(5) & 0xFF == ord('q'):
# 		break

# # When all the process is done
# # Release the capture and destroy all windows
# capture.release()
# cv2.destroyAllWindows()

In [ ]:
# # with different colors
# import cv2
# import mediapipe as mp
# import time
# import numpy as np

# # Initializing mediapipe modules
# mp_holistic = mp.solutions.holistic
# mp_drawing = mp.solutions.drawing_utils

# # Initialize holistic model
# holistic_model = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# # (0) in VideoCapture is used to connect to your computer's default camera
# capture = cv2.VideoCapture(0)

# # Initializing current time and previous time for calculating the FPS
# previousTime = 0

# # Define a list of 21 different colors for each hand landmark
# colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255), 
#           (0, 255, 255), (128, 0, 0), (0, 128, 0), (0, 0, 128), (128, 128, 0),
#           (128, 0, 128), (0, 128, 128), (255, 128, 0), (255, 0, 128), (128, 255, 0),
#           (0, 255, 128), (128, 0, 255), (0, 128, 255), (255, 128, 128), (128, 255, 128),
#           (128, 128, 255)]

# while capture.isOpened():
#     # Capture frame by frame
#     ret, frame = capture.read()

#     # Resizing the frame for better view
#     frame = cv2.resize(frame, (800, 600))

#     # Converting the from BGR to RGB
#     image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

#     # Making predictions using holistic model
#     image.flags.writeable = False
#     results = holistic_model.process(image)
#     image.flags.writeable = True

#     # Converting back the RGB image to BGR
#     image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

#     # If landmarks are detected on the right hand
#     if results.right_hand_landmarks:
#         # Loop through each landmark point
#         for idx, landmark in enumerate(results.right_hand_landmarks.landmark):
#             # Get landmark coordinates
#             h, w, c = image.shape
#             cx, cy = int(landmark.x * w), int(landmark.y * h)

#             # Draw circle for each landmark with a different color
#             cv2.circle(image, (cx, cy), 7, colors[idx], cv2.FILLED)

#         # Optionally, draw hand connections (in a single color if desired)
#         mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

#     # Similarly, you can do this for the left hand
#     if results.left_hand_landmarks:
#         for idx, landmark in enumerate(results.left_hand_landmarks.landmark):
#             h, w, c = image.shape
#             cx, cy = int(landmark.x * w), int(landmark.y * h)
#             cv2.circle(image, (cx, cy), 7, colors[idx], cv2.FILLED)

#         mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

#     # Calculating the FPS
#     currentTime = time.time()
#     fps = 1 / (currentTime - previousTime)
#     previousTime = currentTime

#     # Displaying FPS on the image
#     cv2.putText(image, str(int(fps)) + " FPS", (10, 70), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)

#     # Display the resulting image
#     cv2.imshow("Hand Landmarks with Different Colors", image)

#     # Enter key 'q' to break the loop
#     if cv2.waitKey(5) & 0xFF == ord('q'):
#         break

# # Release the capture and destroy all windows
# capture.release()
# cv2.destroyAllWindows()


In [ ]:
# # hands with numbers
# import cv2
# import time
# import mediapipe as mp

# # Initialize MediaPipe components
# mp_holistic = mp.solutions.holistic
# mp_drawing = mp.solutions.drawing_utils

# # Create a holistic model instance
# holistic_model = mp_holistic.Holistic()

# # (0) in VideoCapture is used to connect to your computer's default camera
# capture = cv2.VideoCapture(0)

# # Initializing current time and previous time for calculating the FPS
# previousTime = 0
# currentTime = 0

# while capture.isOpened():
#     # Capture frame by frame
#     ret, frame = capture.read()

#     # Resizing the frame for better view
#     frame = cv2.resize(frame, (800, 600))

#     # Converting the frame from BGR to RGB
#     image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

#     # Making predictions using holistic model
#     image.flags.writeable = False
#     results = holistic_model.process(image)
#     image.flags.writeable = True

#     # Converting back the RGB image to BGR
#     image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

#     # Drawing Right hand Landmarks
#     if results.right_hand_landmarks:
#         mp_drawing.draw_landmarks(
#             image,
#             results.right_hand_landmarks,
#             mp_holistic.HAND_CONNECTIONS
#         )
#         # Adding landmark numbers to right hand
#         for id, lm in enumerate(results.right_hand_landmarks.landmark):
#             h, w, _ = image.shape
#             cx, cy = int(lm.x * w), int(lm.y * h)
#             cv2.circle(image, (cx, cy), 5, (0, 0, 255), -1)  # Red circle for landmarks
#             cv2.putText(image, str(id), (cx, cy - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)  # Label each landmark

#     # Drawing Left hand Landmarks
#     if results.left_hand_landmarks:
#         mp_drawing.draw_landmarks(
#             image,
#             results.left_hand_landmarks,
#             mp_holistic.HAND_CONNECTIONS
#         )
#         # Adding landmark numbers to left hand
#         for id, lm in enumerate(results.left_hand_landmarks.landmark):
#             h, w, _ = image.shape
#             cx, cy = int(lm.x * w), int(lm.y * h)
#             cv2.circle(image, (cx, cy), 5, (0, 0, 255), -1)  # Red circle for landmarks
#             cv2.putText(image, str(id), (cx, cy - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1)  # Label each landmark

#     # Calculating the FPS
#     currentTime = time.time()
#     fps = 1 / (currentTime - previousTime)
#     previousTime = currentTime

#     # Displaying FPS on the image
#     cv2.putText(image, str(int(fps)) + " FPS", (10, 70), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)

#     # Display the resulting image
#     cv2.imshow("Facial and Hand Landmarks", image)

#     # Enter key 'q' to break the loop
#     if cv2.waitKey(5) & 0xFF == ord('q'):
#         break

# # When all the process is done
# # Release the capture and destroy all windows
# capture.release()
# cv2.destroyAllWindows()


In [ ]:
# Code to access landmarks
for landmark in mp_holistic.HandLandmark:
    print(landmark, landmark.value)
 
print(mp_holistic.HandLandmark.WRIST.value)

In [ ]:
# ## STEP 1: Import the necessary modules.
# import mediapipe as mp
# from mediapipe.tasks import python
# from mediapipe.tasks.python import vision

# STEP 2: Create an HandLandmarker object.
# base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
# options = vision.HandLandmarkerOptions(base_options=base_options,
#                                        num_hands=2)
# detector = vision.HandLandmarker.create_from_options(options)
# STEP 3: Load the input image.
# image = mp.Image.create_from_file("a1.jpg")

# STEP 4: Detect hand landmarks from the input image.
# detection_result = detector.detect(image)

# STEP 5: Process the classification result. In this case, visualize it.
# annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
# cv2_imshow(cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))##